# Решение на 1.0 на test accuracy. Это не лучшее решение, надо еще перебрать некоторые гиперпараметры, т.к. было показано, что до применения алгоритма максимального паросочетания возможно улучшить validation accuracy существенно - с 92 до 95.

In [1]:
import re
import os
import gc
import pickle
import itertools

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn import preprocessing
from sklearn import decomposition

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

from tqdm import tqdm

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

ModuleNotFoundError: No module named 'pymorphy2'

# Важная информация: иногда при обучении той или иной штуки могут происходить странные вещи, не относящиеся к нашему коду. Иногда может вылетать KeyError, иногда падает kernel...

### Чистка текста:
#### 1) Оставить только буквы
#### 2) Слова длинее одного символа
#### 3) Выкинуть стоп-слова
#### 4) Лемматизация (приведение к начальной форме)

In [3]:
stop_words = ["а","е","и","ж","м","о","на","не","ни","об","но","он","мне","мои","мож","она","они","оно","мной","много","многочисленное","многочисленная","многочисленные","многочисленный","мною","мой","мог","могут","можно","может","можхо","мор","моя","моё","мочь","над","нее","оба","нам","нем","нами","ними","мимо","немного","одной","одного","менее","однажды","однако","меня","нему","меньше","ней","наверху","него","ниже","мало","надо","один","одиннадцать","одиннадцатый","назад","наиболее","недавно","миллионов","недалеко","между","низко","меля","нельзя","нибудь","непрерывно","наконец","никогда","никуда","нас","наш","нет","нею","неё","них","мира","наша","наше","наши","ничего","начала","нередко","несколько","обычно","опять","около","мы","ну","нх","от","отовсюду","особенно","нужно","очень","отсюда","в","во","вон","вниз","внизу","вокруг","вот","восемнадцать","восемнадцатый","восемь","восьмой","вверх","вам","вами","важное","важная","важные","важный","вдали","везде","ведь","вас","ваш","ваша","ваше","ваши","впрочем","весь","вдруг","вы","все","второй","всем","всеми","времени","время","всему","всего","всегда","всех","всею","всю","вся","всё","всюду","г","год","говорил","говорит","года","году","где","да","ее","за","из","ли","же","им","до","по","ими","под","иногда","довольно","именно","долго","позже","более","должно","пожалуйста","значит","иметь","больше","пока","ему","имя","пор","пора","потом","потому","после","почему","почти","посреди","ей","два","две","двенадцать","двенадцатый","двадцать","двадцатый","двух","его","дел","или","без","день","занят","занята","занято","заняты","действительно","давно","девятнадцать","девятнадцатый","девять","девятый","даже","алло","жизнь","далеко","близко","здесь","дальше","для","лет","зато","даром","первый","перед","затем","зачем","лишь","десять","десятый","ею","её","их","бы","еще","при","был","про","процентов","против","просто","бывает","бывь","если","люди","была","были","было","будем","будет","будете","будешь","прекрасно","буду","будь","будто","будут","ещё","пятнадцать","пятнадцатый","друго","другое","другой","другие","другая","других","есть","пять","быть","лучше","пятый","к","ком","конечно","кому","кого","когда","которой","которого","которая","которые","который","которых","кем","каждое","каждая","каждые","каждый","кажется","как","какой","какая","кто","кроме","куда","кругом","с","т","у","я","та","те","уж","со","то","том","снова","тому","совсем","того","тогда","тоже","собой","тобой","собою","тобою","сначала","только","уметь","тот","тою","хорошо","хотеть","хочешь","хоть","хотя","свое","свои","твой","своей","своего","своих","свою","твоя","твоё","раз","уже","сам","там","тем","чем","сама","сами","теми","само","рано","самом","самому","самой","самого","семнадцать","семнадцатый","самим","самими","самих","саму","семь","чему","раньше","сейчас","чего","сегодня","себе","тебе","сеаой","человек","разве","теперь","себя","тебя","седьмой","спасибо","слишком","так","такое","такой","такие","также","такая","сих","тех","чаще","четвертый","через","часто","шестой","шестнадцать","шестнадцатый","шесть","четыре","четырнадцать","четырнадцатый","сколько","сказал","сказала","сказать","ту","ты","три","эта","эти","что","это","чтоб","этом","этому","этой","этого","чтобы","этот","стал","туда","этим","этими","рядом","тринадцать","тринадцатый","этих","третий","тут","эту","суть","чуть","тысяч"]
letters = 'ёйцукенгшщзхъфывапролджэячсмитьбю qwertyuiopasdfghjklzxcvbnm'


# функция отчистки одного инстанса текста
def clean(string):
    string = string.lower()
    string = ''.join(c for c in string if c in letters)
    spliced = string.split(' ')
    string = ''.join((morph.parse(word)[0]).normal_form + ' ' for word in spliced if len(word) > 1 and not word in stop_words)
    return string[:-1]

### Чистка текста:
#### 1) Оставить только буквы
#### 2) Слова длинее одного символа
#### 3) Выкинуть стоп-слова
#### 4) Лемматизация (приведение к начальной форме)

# Чистка данных для обучения.
### Чистка занимает минут 15.

In [4]:
df = pd.read_csv('Task_1/train/train.csv')
N = len(df)
print("Train data size: {}".format(N))
df.head()

Train data size: 3485


,author,text
0,__label__34,[стр. 11 — 29 бумажной версии номера] Игорь Па...
1,__label__50,Мелочи жизни Проплаченная культура Конец мин...
2,__label__6,Юношеское Между низшими и высшими не закончен ...
3,__label__12,"Свет, упавший из тишины… — родился и живёт..."
4,__label__58,ПОЛЕМИКА * ЧЕРНОСОТЕНСТВО: ПРОШЛОЕ И ПЕРСПЕК...


In [ ]:
for i in tqdm(range(N)):
    df.iloc[i]['text'] = clean(df.iloc[i]['text'])

df.to_csv('./Data/train_1_clean.csv', sep=';', encoding='utf-8')
df.sample(frac=1, random_state=42).head()

# Чистка данных для валидации.
### Чистка занимает минут 5.

In [5]:
def create_clean_test_filename(i):
    return "./Data/test_1_clean/test_" + str(i) + "_clean.csv"

In [ ]:
for i in tqdm(range(15, 20)):
    df = pd.read_csv('./Data/valid_texts/text_' + str(i) + '.csv')
    df["text"] = df["text"].apply(clean)
    df.to_csv(create_clean_test_filename(i), sep=';', encoding='utf-8')

# Непосредственно модель.

In [6]:
train_df = pd.read_csv("./Data/train_1_clean.csv", sep=";")
test_dfs = [pd.read_csv(create_clean_test_filename(i), sep=';', index_col=0) for i in range(15, 20)]

y = [int(string[9:]) for string in train_df.author.values] # remove "__label__" from string and convert to int

# делим на train и valid выборки
xtrain, xvalid, ytrain, yvalid = train_test_split(train_df.text.values, y,
                                                  stratify=train_df["author"],
                                                  random_state=42,
                                                  test_size=0.1,
                                                  shuffle=True)

xtest = [test_df.text.values for test_df in test_dfs]

### Векторизация TF-IDF
# Минуты 3

In [7]:
tfv = TfidfVectorizer(min_df=3, # по этому параметру хорошо бы сделать неплохой GridSearch
                      analyzer='word',
                      token_pattern=r'\w{1,}',
                      ngram_range=(1, 3), # тут тоже можно поиграться
                      use_idf=1,
                      smooth_idf=1,
                      sublinear_tf=1)

all_text_data = list(xtrain) + list(xvalid)
for x in xtest:
    all_text_data += list(x)

tfv.fit(all_text_data)

del all_text_data
gc.collect();

In [8]:
with open('model.pickle', 'wb') as pickle_file:
    pickle.dump(tfv, pickle_file, protocol=pickle.HIGHEST_PROTOCOL)

del tfv
gc.collect();

In [9]:
gc.collect();

with open('model.pickle', 'rb') as pickle_file:
    tfv = pickle.load(pickle_file)

In [10]:
xtrain_tfv = tfv.transform(xtrain)
xvalid_tfv = tfv.transform(xvalid)
xtest_tfv = [tfv.transform(x) for x in xtest]

In [11]:
del tfv
gc.collect();

### Обучение логистической регрессии 

In [12]:
log_regr = LogisticRegression(C=1000, max_iter=500, penalty='l2', solver='liblinear', multi_class='auto')
log_regr.fit(xtrain_tfv, ytrain)

LogisticRegression(C=1000, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=500, multi_class='auto',
          n_jobs=None, penalty='l2', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)

In [13]:
y_pred_train = log_regr.predict(xtrain_tfv)
y_pred_valid = log_regr.predict(xvalid_tfv)
y_pred_test = [log_regr.predict(x) for x in xtest_tfv]

train_accuracy = metrics.accuracy_score(ytrain, y_pred_train)
valid_accuracy = metrics.accuracy_score(yvalid, y_pred_valid)

print(train_accuracy, valid_accuracy)

1.0 0.9283667621776505


## Венгерский алгоритм
### Позволяет найти минимальное по весу паросочетание среди максимальных по размеру
### Используется для поиска биекции, максимизирующей вероятность правильности всех ответов

In [14]:
def pos2nneg(a):
    for i in range(len(a)):
        for j in range(len(a[i])):
            a[i][j] = inf if a[i][j]==0 else -np.log(a[i][j])
    return a
def hung(a1):
    n, m = len(a1), len(a1[0])#60, 60
    a = np.zeros((n+1,m+1))
    for i in range(n):
        for j in range(m):
            a[i+1][j+1] = a1[i][j]
    #print(a)
    u = [0]*(n+1)
    v = [0]*(m+1)
    p = [0]*(m+1)
    way = [0]*(m+1)
    inf = np.inf
    for i in range(1, n+1):
        p[0] = i
        j0 = 0
        minv = [inf]*(m+1)
        used = [False]*(m+1)
        while True:
            used[j0] = True
            i0 = p[j0]
            delta = inf
            j1 = -1
            for j in range(1, m+1):
                if not used[j]:
                    cur = a[i0][j]-u[i0]-v[j]
                    if cur < minv[j]:
                        minv[j] = cur
                        way[j] = j0
                    if minv[j] < delta:
                        delta = minv[j]
                        j1 = j
            for j in range(m+1):
                if used[j]:
                    u[p[j]] += delta
                    v[j] -= delta
                else:
                    minv[j] -= delta
            j0 = j1
            if p[j0] == 0:
                break
        while True:
            j1 = way[j0]
            p[j0] = p[j1]
            j0 = j1
            if j0 == 0:
                break
    ans = [-1]*(n)
    for j in range(1,m+1):
        ans[p[j]-1] = j-1
    return ans

## Финальное предсказание для валидации

In [15]:
y_pred_test_probabilities = [log_regr.predict_proba(x) for x in xtest_tfv]

for probability_matrix in y_pred_test_probabilities:
    prediction = hung(pos2nneg(probability_matrix))
    
    print("author")
    for author_id in prediction:
        print("__label__{}".format(author_id))

author
__label__48
__label__21
__label__32
__label__44
__label__40
__label__18
__label__29
__label__36
__label__26
__label__56
__label__22
__label__51
__label__11
__label__27
__label__52
__label__58
__label__23
__label__50
__label__38
__label__3
__label__10
__label__12
__label__34
__label__14
__label__46
__label__0
__label__49
__label__41
__label__28
__label__53
__label__24
__label__4
__label__45
__label__7
__label__15
__label__31
__label__42
__label__57
__label__5
__label__2
__label__16
__label__17
__label__43
__label__30
__label__9
__label__47
__label__59
__label__19
__label__37
__label__39
__label__54
__label__55
__label__1
__label__35
__label__6
__label__20
__label__33
__label__8
__label__25
__label__13
author
__label__44
__label__19
__label__57
__label__38
__label__30
__label__4
__label__12
__label__58
__label__51
__label__13
__label__8
__label__47
__label__5
__label__41
__label__24
__label__29
__label__6
__label__49
__label__23
__label__2
__label__53
__label__21
__label__16
__lab